# 1. 라이브러리 및 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette('Set3')
%matplotlib inline

pd.options.display.max_columns = None
pd.options.display.max_rows = 20

import warnings
warnings.filterwarnings('ignore')

### 군집 시각화

In [ ]:
import folium
import matplotlib.colors as mcolors

file_path = '/content/drive/MyDrive/서강대/1학기_빅데이터예측분석/기말고사 프로젝트/df_area.xlsx'

# Load the data again
df_area = pd.read_excel(file_path)

In [ ]:
df_area = df_area.rename(columns = {'cluster76' : 'cluster',
                                    '위도' : 'lat',
                                    '경도' : 'lon'})
df_area.head()

,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,자치구_코드,자치구_코드_명,행정동_코드,행정동_코드_명,영역_면적,lat,lon,cluster
0,A,골목상권,3110481,불광역 7번,11380,은평구,11380570,대조동,88375,37.615007,126.924526,1
1,A,골목상권,3110455,역촌노인복지관,11380,은평구,11380625,역촌동,155405,37.606204,126.910367,1
2,A,골목상권,3110456,구산보건지소,11380,은평구,11380560,구산동,170875,37.610547,126.910883,1
3,A,골목상권,3110457,예일여고,11380,은평구,11380560,구산동,93412,37.614467,126.910969,1
4,A,골목상권,3110484,역촌역 3번,11380,은평구,11380510,녹번동,191591,37.609016,126.924951,1


In [ ]:
# 이태원, 경리단길, 한강진, 한남 포함 상권 코드명 출력
print(df_area[df_area['cluster'] == 23]['상권_코드_명'])

459         용산구청
460      서빙고역 1번
461       경리단길북측
462       경리단길남측
463      이태원역 북측
         ...    
477    이태원(이태원역)
478        한남오거리
479        이태원시장
480         보광시장
481     이태원 관광특구
Name: 상권_코드_명, Length: 23, dtype: object


- 군집 전 상권 분포

In [ ]:
import folium

# 지도의 중심을 설정하기 위한 평균 좌표 계산
mean_lat = df_area['lat'].mean()
mean_lon = df_area['lon'].mean()

# 지도 생성 (첫 번째 위치를 중심으로 설정)
map_before_cluster = folium.Map(location=[mean_lat, mean_lon], zoom_start=11)

# 각 위치에 대한 마커 추가
for index, row in df_area.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=3, # 원의 크기
        color='blue', # 원의 색상
        fill=True,
        fill_color='blue', # 원 내부의 색상
        fill_opacity=0.5 # 원 내부 색상의 불투명도
    ).add_to(map_before_cluster)

# 배경지도 타일 설정하기
layer = "Base"
tileType = "png"
vworld_key = "711829AA-29D7-3495-808F-7DD27874AD23"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(map_before_cluster)

map_before_cluster

- 군집 후 상권 분포

In [ ]:
# Generating a color map for the clusters
# Using a set of distinct colors from matplotlib
colors = list(mcolors.TABLEAU_COLORS.values())
num_clusters = len(df_area['cluster'].unique())
color_map = {cluster: colors[i % len(colors)] for i, cluster in enumerate(range(1, num_clusters + 1))}

# Create a map centered around the average latitude and longitude
map_center_lat = df_area['lat'].mean()
map_center_lon = df_area['lon'].mean()
map = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=11)

# Add circles to the map
for _, row in df_area.iterrows():
    folium.Circle(
        location=[row['lat'], row['lon']],
        radius=100,  # radius in meters
        color=color_map[row['cluster']],
        fill=True,
        fill_color=color_map[row['cluster']],
        popup=f'Cluster: {row["cluster"]}'  # Popup to show when circle is clicked
      ).add_to(map)

# 배경지도 타일 설정하기
layer = "Base"
tileType = "png"
vworld_key = "711829AA-29D7-3495-808F-7DD27874AD23"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(map)

# Show the map (in Jupyter, this will display the map inline)
map


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 각 군집의 중심점 계산
centroids = df_area.groupby('cluster').agg({'lat': 'mean', 'lon': 'mean'}).reset_index()
centroids
centroids.head()

,cluster,lat,lon
0,1,37.608705,126.918894
1,2,37.569317,126.813573
2,3,37.549983,127.097222
3,4,37.494159,126.990358
4,5,37.504565,127.090196


In [ ]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

k=76

# 군집 개수에 따른 색상 맵 생성
color_map = cm.get_cmap('hsv', k)

# 지도의 중심을 설정하기 위한 평균 좌표 계산
mean_lat = df_area['lat'].mean()
mean_lon = df_area['lon'].mean()

# 지도 생성
map_cluster = folium.Map(location=[mean_lat, mean_lon], zoom_start=11)

# 각 군집에 대해 다른 색상의 원을 추가
for cluster_num in range(k):
    cluster_data = df_area[df_area['cluster'] == cluster_num]
    cluster_center = [centroids.loc[cluster_num, 'lat'], centroids.loc[cluster_num, 'lon']]
    cluster_size = len(cluster_data)

    # 색상 할당
    cluster_color = colors.to_hex(color_map(cluster_num))

    # 원 추가 (경계선 없음, 흐릿한 표현)
    folium.Circle(
        location=cluster_center,
        radius=1000,  # 원의 크기는 군집 크기에 따라 조정
        color=None,  # 경계선 제거
        fill=True,
        fill_color=cluster_color,
        fill_opacity=0.5  # 투명도 조절
    ).add_to(map_cluster)

# 배경지도 타일 설정하기
layer = "Base"
tileType = "png"
vworld_key = "711829AA-29D7-3495-808F-7DD27874AD23"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(map_cluster)

# 지도 표시
map_cluster